In [10]:
import sys
import numpy as np
from math import exp , sqrt , log , pi
import scipy.linalg as linalg
from scipy.stats import norm
import matplotlib . pyplot as plt

# Explicit and Implicit Finite Difference

The code below implements both the Explicit and Implicit Finite Difference methods to price European Call and Put options.

In [12]:
class Finite(object):
    def __init__(self, S,K,r,T,sigma,Smax,M,N, call):
        self.S=S
        self.K=K
        self.r=r
        self.T=T
        self.sigma=sigma
        self.Smax=Smax
        #self.dS=dS
        self.M, self.N= int(M), int(N)
        self.call=call
        
        self.dS=Smax/float(self.M)
        #self.Smax=dS*float(self.M)
        self.dt=T/float(self.N)
        self.i_values=np.arange(self.M)
        self.j_values=np.arange(self.N)
        self.grid=np.zeros(shape=(self.M+1, self.N+1))
        self.boundary=np.linspace(0,self.Smax,self.M+1)
    def _setup_boundary_conditions_(self):
        pass
    def _setup_coefficients_(self):
        pass
    def _traverse_grid_(self):
        #Iterate grid backward in time
        pass
    def _interpolate_(self):
        #get closest price at S0 with using piecewise linear 
        #interpolation on the initial grid column
        return np.interp(self.S,self.boundary,self.grid[:,0])
    def price(self):
        self._setup_boundary_conditions_()
        self._setup_coefficients_()
        self._traverse_grid_()
        return self._interpolate_()

class ExplicitEU(Finite):
    def _setup_boundary_conditions_(self):
        if self.call:
            self.grid[:,-1]=np.maximum(self.boundary-self.K,0)
            self.grid[-1,:-1]=(self.Smax-self.K)* \
                              np.exp(-self.r*self.dt*
                                     (self.N-self.j_values))
        else:
            self.grid[:,-1]= \
                np.maximum(self.K-self.boundary,0)
            self.grid[0,:-1]=(self.K-self.Smax) * \
                             np.exp(-self.r *
                                    self.dt *
                                    (self.N-self.j_values))
                             
    def _setup_coefficients_(self):
        self.a=0.5*self.dt*((self.sigma**2) *
                            (self.i_values**2) - 
                            self.r*self.i_values)
        self.b=1-self.dt*((self.sigma**2) *
                          (self.i_values**2) +
                          self.r)
        self.c=0.5*self.dt*((self.sigma**2) *
                            (self.i_values**2) +
                            self.r*self.i_values)
    
    def _traverse_grid_(self):
        for j in reversed(self.j_values):
            for i in range(self.M)[2:]:
                self.grid[i,j]=self.a[i]*self.grid[i-1,j+1] + \
                               self.b[i]*self.grid[i,j+1] + \
                               self.c[i]*self.grid[i+1,j+1]
                        

class ImplicitEU(ExplicitEU):
    def _setup_coefficients_(self):
        self.a=0.5*(self.r*self.dt*self.i_values-
                    (self.sigma**2)*self.dt*(self.i_values**2))
        self.b=1 + \
               (self.sigma**2)*self.dt*(self.i_values**2) + \
               self.r*self.dt
        self.c=-0.5*(self.r*self.dt*self.i_values+
                     (self.sigma**2)*self.dt*(self.i_values**2))
        self.coeffs= (np.diag(self.a[2:self.M],-1) + \
                     np.diag(self.b[1:self.M]) +
                     np.diag(self.c[1:self.M-1],1))

    def _traverse_grid_(self):
       P, L, U=linalg.lu(self.coeffs)
       aux=np.zeros(self.M-1)                        

       
       for j in reversed(range(self.N)):
           aux[0]=np.dot(-self.a[1], self.grid[0,j])
           x1=linalg.solve(L,self.grid[1:self.M, j+1]+aux)
           x2=linalg.solve(U,x1)
           self.grid[1:self.M, j]=x2
       
                       
           
option=ExplicitEU(100,100,0.03,1,0.25,150,30,313, False)
print(option.price())                      
#                              
option=ImplicitEU(100,100,0.03,1,0.25,150,30,313, False)        
print(option.price()) 


8.344696829578778
8.336160485072119


## Alternative solution

The script below is a more simple approach to implementing the Explicit and Implicit Finite Difference methods for pricing European Call and Put options.

It also calculates some of the *Greeks* (Delta, gamma, theta, vega). 

In [5]:
def e_explicit(K,T,S,sigma,r,q,N,M,dx,call):
    dt=T/N
    nu=r-q-0.5*sigma**2
    edx=exp(dx)
    pu=0.5*dt*((sigma/dx)**2+nu/dx)
    pm=1-dt*(sigma/dx)**2-r*dt
    pd=0.5*dt*((sigma/dx)**2-nu/dx)
    St=[0]*(2*M+1)
    C=[[0 for j in range(2*M+1)] for i in range(N+1)]
    #Initialise asset prices at maturity
    St[0]=S*exp(-M*dx)
    for j in range(1, 2*M+1):
        St[j]=St[j-1]*edx
    #Option Values at maturity
    for j in range(0, 2*M+1):
        if call==True:
            C[N][j]=max(0,St[j]-K)
        elif call==False:
            C[N][j]=max(0,K-St[j])
    #Stepping back in time:
    for i in range(N-1,-1,-1):
        for j in range(1, 2*M):
            C[i][j]=pu*C[i+1][j+1]+pm*C[i+1][j]+pd*C[i+1][j-1]
#    #Boundary conditions:
        if call==True:
           C[i][0]=C[i][1]
           C[i][2*M]=C[i][2*M-1]+(St[2*M]-St[2*M-1])
        elif call==False:
            C[i][0]=C[i][1]+(St[1]-St[0])
            C[i][2*M]=C[i][2*M-1]
            
    delta= (C[0][M+1]-C[0][M-1])/(St[M+1]-St[M-1])
    gamma= (((C[0][M+1]-C[0][M])/(St[M+1]-St[M]))-
              ((C[0][M]-C[0][M-1])/(St[M]-St[M-1]))) /(0.5*(St[M+1]-St[M-1]))
    theta= (C[1][M]-C[0][M])/dt
    return C[0][M], delta, gamma, theta
print(e_explicit(100,1,100,0.25,0.06,0.03,1190,60,0.012,True))   

(11.011918739913476, 0.5792139969717195, 0.015032450205724234, -5.777066060407119)


In [6]:
def Vega(sigma):
    delt=0.0001
    vega=((e_explicit(100,1,100,sigma+delt,0.06,0.03,600,300,0.02,False)[0]-
      e_explicit(100,1,100,sigma-delt,0.06,0.03,600,300,0.02,False)[0])/(200*delt))
    return vega

The code below implements the Implicit Finite Difference method to price both European Call and Put options.

In [11]:
def e_implicit(K,T,S,sigma,r,q,N,M,dx,call):
   dt=T/N
   nu=r-q-0.5*sigma**2
   edx=exp(dx)
   pu=-0.5*dt*((sigma/dx)**2+nu/dx)
   pm=1+dt*(sigma/dx)**2+r*dt
   pd=-0.5*dt*((sigma/dx)**2-nu/dx)
   #initialise asset prices at maturity:
   St=[0]*(2*M+1)
   #C=[[0 for j in range(2*M+1)] for i in range(N+1)]
   C=np.zeros(shape=(N+1, 2*M+1))
   St[0]=S*exp(-M*dx)
   #C=np.zeros(shape=(N+1, 2*M+1))
   for j in range(1, 2*M+1):
       St[j]=St[j-1]*edx
   #Option Values at maturity:
   for j in range(0, 2*M+1):
       if call==True:
           C[N,j]=max(0,St[j]-K)
       elif call==False:
           C[N,j]=max(0,K-St[j])
#    #Boundary conditions:
#    for i in range(N-1,-1,-1):
#        if call==True:
#           C[i,0]=C[i,1]
#           #d=[St[2*M]-St[2*M-1], C[i+1]]
#           C[i,2]=C[i+1,1]-pm*C[i,1]
#           C[i][2*M]=C[i][2*M-1]+(St[2*M]-St[2*M-1])
#        elif call==False:
#            C[i][0]=C[i][1]+(St[1]-St[0])
#            C[i][2*M]=C[i][2*M-1]

   d=np.insert(C[:,N+1], 0, (St[2*M]-St[2*M-1]))
   d=np.append(d,1)

   c=[pu]*(2*M-1)
   c=np.insert(c,0,-1)
   b=[pm]*(2*M-1)
   b=np.insert(b,0,1)
   b=np.append(b,-1)
   a=[pd]*(2*M-1)
   a=np.append(a,1)

   return len(a)

print(e_implicit(100,1,100,0.2,0.03,0.0,3,6,0.2,True))    


12


# Crank-Nicolson Finite Difference method

The script below can be used to price European Call and Put options using the Crank-Nicolson method.

In [13]:
class CrankEU(ExplicitEU):

    def _setup_coefficients_(self):
        self.alpha = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) -
            self.r*self.i_values)
        self.beta = -self.dt*0.5*(
            (self.sigma**2)*(self.i_values**2) +
            self.r)
        self.gamma = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) +
            self.r*self.i_values)
        self.M1 = -np.diag(self.alpha[2:self.M], -1) + \
                  np.diag(1-self.beta[1:self.M]) - \
                  np.diag(self.gamma[1:self.M-1], 1)
        self.M2 = np.diag(self.alpha[2:self.M], -1) + \
                  np.diag(1+self.beta[1:self.M]) + \
                  np.diag(self.gamma[1:self.M-1], 1)

    def _traverse_grid_(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.M1)

        for j in reversed(range(self.N)):
            x1 = linalg.solve(L,
                              np.dot(self.M2,
                                     self.grid[1:self.M, j+1]))
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2

option=CrankEU(100,100,0.03,1,0.25,150,90,60, False)        
print(option.price())


8.384101905803645
